In [1]:
import pandas as pd
import openai
from dotenv import load_dotenv
load_dotenv()
import os
from langchain.prompts import load_prompt
from langchain import PromptTemplate
from langchain.llms import OpenAIChat
import sys
pd.options.mode.chained_assignment = None
from langchain.callbacks import get_openai_callback
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    SystemMessage
)
import sys
sys.path.append('./modules/')
openai.api_key = os.getenv('OPENAI_KEY')
key = os.getenv('OPENAI_KEY')

from Generator import Generator
from Extractor import Extractor
from PromptPath import PromptPath
from GenExtract import GenExtract
from Property import Property
from NaturalLanguagePool import NaturalLanguagePool
from DynamicChatHistory import DynamicChatHistory
from Conversation import Conversation
from ConversationScript import ConversationScript
from ConversationScriptStage import ConversationScriptStage
from show_chat_log import show_chat_log
from is_question import is_question
from customer_profile import customer_profile

times_embedding_run = 0

#### Step 1: Create a directory in the prompts folder to store the contents. Then save the path

In [2]:
#prompt folder_name
prompt_folder_name = 'customer_issues_extractor'

#check to see if the the folder name exists in the prompts folder
if prompt_folder_name in os.listdir('prompts'):
    print('prompt folder exists')
    prompt_folder_path = 'prompts/' + prompt_folder_name
else:
    os.mkdir('prompts/' + prompt_folder_name)
    prompt_folder_path = 'prompts/' + prompt_folder_name

path = PromptPath(prompt_folder_path + '/', type="json")

def save(prompt, name):
    prompt.save(prompt_folder_path + '/' + name + '.json')

prompt folder exists


#### Step 2: Create your base prompt & Test It

In [3]:
text = """Summarize the issue that the customer is having. Only include issues that the customer is obviously experiencing based on the customer message. You need to collect the following information: the target pests they are having an issues with as a comma seperated list. Target pest include: ants, spiders, termites, rats, mice, roaches, fleas, ticks, bed bugs, bees, wasps, millipedes, centipedes, crickets, earwigs, silverfish, and moths. If it does not fit any one of them, write None.
Examples
Customer Response: I have a problem with ants in my kitchen.
Pests: Ants
Customer Reponse: Termites and rodents are eating my house.
Pests: Termites, rodents
Customer Response: No issue with rodents
Pests: None
Customer Response: {customer_response}"""
prompt = PromptTemplate(template=text, input_variables=["customer_response"])

save(prompt, 'base_prompt_3_avoid_negatives')

#### Step 3: Create the first version of the generator

In [4]:
model = GenExtract(
    Generator(path('base_prompt')),
    Extractor(type="natural_language_to_dict", keys=["pests"]),
    print_intermediate=True
)

In [5]:
model.run('I have been seeing roaches in the kids room')

Generation:  Pests: Roaches


{'pests': 'Roaches'}

In [34]:
model.run('I am seeing termites and spiders in my house')

Generation:  Pests: Termites, spiders
Extraction:  {'pests': 'Termites, spiders'}


{'pests': 'Termites, spiders'}

In [35]:
model.run('Roaches')

Generation:  

Pests: Roaches
Extraction:  {'pests': 'Roaches'}


{'pests': 'Roaches'}

In [37]:
model.run("No I haven't seen any rodents")

Generation:  Pests: None mentioned or specified.
Extraction:  {'pests': 'None mentioned or specified.'}


{'pests': 'None mentioned or specified.'}

#### Step 4: Switch out the generator and prompt using the following code format

`model.generator = Generator(path('new_prompt'))`

`model.run('Test Case')`

In [28]:
text_two = """Decide which issue the customer is having, then map it to one of the ids below. Only include issues that the customer is obviously experiencing based on the customer message. Then include a summary of any special instructions or target details for the technician. Then, include the location of the issue. Then, collect the square footage of the home.
0: None
1: Ants
2: Spiders
3: Termites
4: Rats
5: Mice
6: Roaches
7: Fleas
8: Ticks
9: Bed Bugs
10: Bees
11: Wasps
12: Millipedes
13: Centipedes
14: Crickets
15: Earwigs
Use 0 if you the customer's issues do not fit any of the above.
Examples
Customer Response: I have a problem with ants in my kitchen.
Pests: Ants
Pest_Id: 1
Special_Instructions: Ants are in the kitchen
Location: Kitchen
Square_Footage: None
Customer Reponse: I think I might have a rodent and termite problem
Pests: Termites, rodents
Pest_Id: 3, 4
Special Instructions: None
Location: None
Square_Footage: None
Customer Response: No issue with rodents
Pests: None
Pest_Id: 0
Special_Instructions: None
Location: None
Square_Footage: None
Customer Response: I need you to call me the morning of the appointment to confirm.
Pests: None
Pest_Id: 0
Special_Instructions: Call the customer the morning of the appointment to confirm
Location: None
Square_Footage: None
Customer Response: My home is 2000 square feet.
Pests: None
Pest_Id: 0
Special_Instructions: None
Location: None
Square_Footage: 2000
Customer Response: {customer_response}"""
prompt = PromptTemplate(template=text_two, input_variables=["customer_response"])

save(prompt, 'map_summarize_location_size')

In [29]:
model = GenExtract(
    Generator(path('map_summarize_location_size')),
    Extractor(type="natural_language_to_dict", keys=["pests", "pest_id", "special_instructions", "location", "square_footage"]),
    print_intermediate=True
)

In [30]:
model.run('My home is 800 square feet.')

Generation:  

Pests: None 
pest_id: 0 
Special_Instructions: None 
Location: None 
Square_Footage: 800


{'pests': None,
 'pest_id': '0',
 'special_instructions': None,
 'location': None,
 'square_footage': '800'}

In [31]:
model.run('I have been seeing roaches in the kids room')

Generation:  Pests: Roaches
pest_id: 6
Special_Instructions: Roaches are in the kids room
Location: Kids room
Square_Footage: None


{'pests': 'Roaches',
 'pest_id': '6',
 'special_instructions': 'Roaches are in the kids room',
 'location': 'Kids room',
 'square_footage': None}

In [32]:
model.run('I am seeing termites and rodents')

Generation:  

Pests: Termites, rodents
pest_id: 3, 4
Special_Instructions: None
Location: None
Square_Footage: None


{'pests': 'Termites, rodents',
 'pest_id': '3, 4',
 'special_instructions': None,
 'location': None,
 'square_footage': None}

In [33]:
model.run("""
Hi! My name is George from Better Termite & Pest Control. How can I help you? :)
User: Hi name is John Smith and I am interested in pest control
Agent: Hello John! Thank you for your interest in Better Termite & Pest Control. Before we proceed, may I ask a few questions to better understand your pest control needs?
User: Yeah sure, but before we start, do you offer squirrel control?
Agent: I'm sorry, but we don't cover any type of wildlife, including squirrels. However, we do have a range of effective pest control solutions for other common household pests. To better assist you, may I have your email, phone number, street address, city, state and zip code, please?
User: No worries. I also have an ant problem in my kitchen
Agent: We can certainly help with that! Our company offers a range of effective pest control solutions for common household pests, including ants. Before we get started, can I have your contact information, including your first and last name, phone number, email, street address, city, state, and zip code?
User: Sure my name is George Schulz. My email is georgeschulz33@gmail.com, phone is 571-431-9531, and my address is 123 Elm Way, Gaithersburg, MD 20878
Agent: Thank you for providing your contact information, George. Can you please tell me what type of pest issue you are experiencing and where it is located? This will help us better understand your needs and provide the right solution. Additionally, could you please let me know the severity of the issue, the type of service you're interested in, any special instructions, and any time constraints you may have?
""")

Generation:  Pests: Ants
pest_id: 1
Special_Instructions: Ants are in the kitchen
Location: Kitchen
Square_Footage: None


{'pests': 'Ants',
 'pest_id': '1',
 'special_instructions': 'Ants are in the kitchen',
 'location': 'Kitchen',
 'square_footage': None}

In [34]:
model.run("""
Hi! My name is George from Better Termite & Pest Control. How can I help you? :)
User: Hi name is John Smith and I am interested in pest control
Agent: Hello John! Thank you for your interest in Better Termite & Pest Control. Before we proceed, may I ask a few questions to better understand your pest control needs?
User: Yeah sure
Agent: We can certainly help with that! Our company offers a range of effective pest control solutions for common household pests, including ants. Before we get started, can I have your contact information, including your first and last name, phone number, email, street address, city, state, and zip code?
User: Sure my name is George Schulz. My email is georgeschulz33@gmail.com, phone is 571-431-9531, and my address is 123 Elm Way, Gaithersburg, MD 20878
Agent: Thank you for providing your contact information, George. Can you please tell me what type of pest issue you are experiencing and where it is located? This will help us better understand your needs and provide the right solution. Additionally, could you please let me know the severity of the issue, the type of service you're interested in, any special instructions, and any time constraints you may have?
User: I have been seeing termites outside my house
Agent: Oh no! Have you been seeing anything else?
User: Yes! I have been seeing a few millipedes in my basement
Agent: Thank you for letting me know. I will add that to your account. 
User: Please make sure to knock on my door before you come in
Agent: Sure and how many square feet is your home?
User: 1400
""")

Generation:  

Pests: Termites, Millipedes
pest_id: 3, 12
Special_Instructions: Knock on the door before entering
Location: Outside the house (Termites), Basement (Millipedes)
Square_Footage: 1400


{'pests': 'Termites, Millipedes',
 'pest_id': '3, 12',
 'special_instructions': 'Knock on the door before entering',
 'location': 'Outside the house',
 'square_footage': '1400'}

In [35]:
model.run("""
Hi! My name is George from Better Termite & Pest Control. How can I help you? :)
User: Hi name is John Smith and I am interested in pest control
Agent: Hello John! Thank you for your interest in Better Termite & Pest Control. Before we proceed, may I ask a few questions to better understand your pest control needs?
""")

Generation:  Pests: None
pest_id: 0
Special_Instructions: None
Location: None
Square_Footage: None


{'pests': None,
 'pest_id': '0',
 'special_instructions': None,
 'location': None,
 'square_footage': None}

In [36]:
model.run("""
User: Hi name is John Smith and I am interested in pest control
Agent: Hey John, thanks for reaching out to us here at Better Termite & Pest Control. I'm happy to assist you with your pest control needs. Do you mind if I ask you a few questions to get a better understanding of your situation?
User: Yeah sure, thats fine
Agent: Great, thanks! Can you kindly provide me with your phone number, email, street address, city, state, and zip code? That way we can get in touch with you and provide a more accurate estimate.
User: Sure my name is George Schulz. My email is georgeschulz33@gmail.com, phone is 571-431-9531, and my address is 123 Elm Way, Gaithersburg, MD 20878
Agent: Thanks for providing that information, George! Can you let me know what type of pests you are having an issue with? That way we can provide the proper treatment.
User: I've been seeing ants in my kitchen and some spiders around the house. Im just fed up!
Agent: I'm sorry to hear that, George! Just to clarify, are you experiencing issues with any other pests besides ants and spiders?
User: No right now, just those two pests.
Agent: Got it, thanks for letting me know! Can you tell me the approximate square footage of your property? It will help us in determining the extent of the problem.
User: It's 2500 square feet
Agent: Thanks for letting me know, George! Based on your description, it sounds like you may have an issue with ants and spiders in your kitchen and around your house. Is that correct? Additionally, have you noticed any other types of pests around your property?
User: Yes, thats right
Agent: Thanks for confirming, George! Just one more question, do you happen to know what type of ants you are seeing in your kitchen? It can help us identify the proper treatment. If you're not sure, that's okay! We have a team of experts who will be able to help identify them.
User: Im not sure
Agent: No problem, George! Our team of experts will be able to identify the ants for you when they come for the inspection. So, just to recap, you are experiencing issues with ants in your kitchen and spiders around your house, correct? And the square footage of your property is 2500. Is there anything else you'd like to share or any other questions you have?
""")

Generation:  Pests: Ants, Spiders
pest_id: 1, 2
Special_Instructions: Ants are in the kitchen and spiders are around the house.
Location: Kitchen and House
Square_Footage: 2500


{'pests': 'Ants, Spiders',
 'pest_id': '1, 2',
 'special_instructions': 'Ants are in the kitchen and spiders are around the house.',
 'location': 'Kitchen and House',
 'square_footage': '2500'}